In [35]:
import os
import re
import pandas as pd
from datetime import datetime as dt

In [37]:
def clear(df):
    df = df.copy()
    df.content = df.content.apply(lambda r: re.sub('[^a-zA-ZáéíñóúüÁÉÍÑÓÚÜ]', ' ', r))
    df.content = df.content.apply(lambda r: re.sub('\s+', ' ', r))
    return df

def filtr(df):
    df = df.copy()
    print(df.shape)
    df['esp'] = df.content.apply(lambda r: re.sub('[^áéíñóúüÁÉÍÑÓÚÜ]', '', r))
    df = df[df.esp != '']
    df.drop('esp', axis=1, inplace=True)
    print(df.shape)
    return df

In [39]:
first_start, start = dt.now(), dt.now()
df = pd.DataFrame(columns=['domain', 'content', 'source', 'lab'])
key_word = 'religion'
for file in os.listdir():
    if file.endswith('.csv') and '_{}'.format(key_word) in file:
        d = pd.read_csv(file, sep='\t')
        if file.startswith('18_'):
            d['source'] = 'google'
        else:
            d['source'] = 'bd'
        df = pd.concat([df, d], axis=0)
df.drop_duplicates('domain', inplace=True)
print("Все файлы собраны и почищены от дубликатов за {}".format(dt.now() - start))
start = dt.now()
df = clear(df)
print("Датасет очищен за {}".format(dt.now() - start))
start = dt.now()
df = filtr(df)
print("Датасет отфильтрован от неиспанских текстов за {}".format(dt.now() - start))
start = dt.now()
df.content = df.content.apply(lambda r: r.lower())
df.to_csv('{}_cont.csv'.format(key_word), sep='\t', index=False)
df_mini = df.copy()
df_mini.content = df_mini.content.apply(lambda r: ' '.join(list(r.split(' '))[:150]))
df_mini.to_csv('{}_cont_mini.csv'.format(key_word), sep='\t', index=False)
print("Вся обработка и сохранение в два файла заняли {}".format(dt.now() - first_start))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


Все файлы собраны и почищены от дубликатов за 0:00:01.986816
Датасет очищен за 0:00:23.384016
(2707, 4)
(2487, 4)
Датасет отфильтрован от неиспанских текстов за 0:00:26.024254
Вся обработка и сохранение в два файла заняли 0:00:59.732943


In [40]:
pattern = ['religion', 'el templo', 'la iglesia', 'sacerdote', 'ateo', 'protestante',
            'catolico', 'musulmán', 'dios', 'la biblia', 'el icono', 'fe', 'Jesús',
            'Buda', 'El Corán', 'la mezquita', 'paraiso', 'Cristo', 'Allah', 'Judío',
            'monje', 'la catedral', 'el monasterio', 'adorar', 'culto', 'secta',
            'Jehová', 'iglesia', 'deidad', 'la cruz', 'predicación', 'santo',
            'agnóstico', 'publicar', 'pecados', 'Masonería', 'patriarca', 'sinagoga',
            'herejía', 'ritual', 'rabino', 'fanático', 'el apóstol', 'celular', 'pastor',
            'ídolo', 'canon', 'laurel', 'crucifijo', 'confesión',]
# pattern = ['abogado', 'acordeón', 'alguacil', 'candidato presidencial', 'codigo',
#          'congreso', 'controversia', 'corrupción', 'criminal', 'código de leyes', 'debate',
#          'declaración', 'decreto presidencial', 'dictadura', 'diputado', 'el derecho',
#          'el embajador', 'el fiscal', 'el ministro', 'el pensamiento', 'el presidente',
#          'factura', 'fiesta', 'fiscalía', 'ideologia', 'jurisprudencia', 'justicia',
#          'juzgar', 'la constitución', 'la corte', 'la cumbre', 'la cárcel', 'la elección',
#          'la ley', 'la oposicion', 'la policia', 'legislación', 'liberal', 'mandato',
#          'monarquia', 'movimiento politico', 'oficial', 'orador', 'parlamento', 'policia',
#          'politica', 'politólogo', 'primer', 'produccion', 'rally', 'republica',
#          'sanciones', 'se desocupa', 'senado', 'trono', 'votar',]

In [41]:
def ranging(df, pattern):
    new_df = df.copy(deep=True)
    new_df['rang'] = new_df.content.\
                apply(lambda r: sum([len(re.findall(p, r)) for p in pattern]))
    return new_df.sort_values('rang', ascending=False)    

In [42]:
new_df = ranging(df, pattern)

In [44]:
df_mini = new_df.copy(deep=True)
df_mini.content = df_mini.content.apply(lambda r: ' '.join(list(r.split(' '))[:150]))
df_mini.to_csv('{}_cont_mini_filtr.csv'.format(key_word), sep='\t', index=False)

In [6]:
set('''Acordeón
Debate
politica
Declaración
se desocupa
liberal
la ley
ideologia
congreso
rally
la oposicion
orador
parlamento
controversia
el presidente
trono
produccion
republica
fiesta
la elección
monarquia
dictadura
votar
movimiento politico
la corte
la constitución
el derecho
juzgar
codigo
abogado
la cárcel
abogado
la policia
legislación
el fiscal
policia
criminal
el pensamiento
decreto presidencial
código de leyes
parlamento
justicia
jurisprudencia
diputado
fiscalía
factura
alguacil
oficial
codigo
el ministro
corrupción
Senado
candidato presidencial
politólogo
el embajador
mandato
primer
sanciones
la oposicion
la cumbre'''.lower().split('\n'))

{'abogado',
 'acordeón',
 'alguacil',
 'candidato presidencial',
 'codigo',
 'congreso',
 'controversia',
 'corrupción',
 'criminal',
 'código de leyes',
 'debate',
 'declaración',
 'decreto presidencial',
 'dictadura',
 'diputado',
 'el derecho',
 'el embajador',
 'el fiscal',
 'el ministro',
 'el pensamiento',
 'el presidente',
 'factura',
 'fiesta',
 'fiscalía',
 'ideologia',
 'jurisprudencia',
 'justicia',
 'juzgar',
 'la constitución',
 'la corte',
 'la cumbre',
 'la cárcel',
 'la elección',
 'la ley',
 'la oposicion',
 'la policia',
 'legislación',
 'liberal',
 'mandato',
 'monarquia',
 'movimiento politico',
 'oficial',
 'orador',
 'parlamento',
 'policia',
 'politica',
 'politólogo',
 'primer',
 'produccion',
 'rally',
 'republica',
 'sanciones',
 'se desocupa',
 'senado',
 'trono',
 'votar'}